In [ ]:
import torch
import torchvision
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from torchvision import datasets
from torchvision.utils import save_image

import skimage 
import math
# import io
# import requests
# from PIL import Image

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import sys
import os


In [ ]:
from cae import *
from helpers import *
from helper_modules import *

Bibliography:

* [Stacked Convolutional Auto-Encoders for Hierarchical Feature Extraction](http://people.idsia.ch/~ciresan/data/icann2011.pdf)
    

In [ ]:
tg = transforms.Grayscale()

In [ ]:
tg?

In [ ]:
# Hyper Parameters
# num_epochs = 5
# batch_size = 100
# learning_rate = 0.001

num_epochs = 100
batch_size = 128
learning_rate = 0.0001

In [ ]:
model = MultiResCAE(12,12,3).cuda()

In [ ]:
model.parameters

In [ ]:
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [ ]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 12, 12)
    return x


In [ ]:
transformation = full_resolution_crop
train_loader, test_loader = get_loaders(batch_size, transformation)

In [ ]:
%%time

for epoch in range(num_epochs):
    for i, (img, labels) in enumerate(train_loader):
        img = Variable(img).cuda()
        # ===================forward=====================
#         print("encoding batch of  images")
        output = model(img)
#         print("computing loss")
        loss = criterion(output, img)
        # ===================backward====================
#         print("Backward ")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        in_pic = to_img(img.cpu().data)
        save_image(pic, './cae_results/2x2-2xfc-out_image_{}.png'.format(epoch))
        save_image(in_pic, './cae_results/2x2-2xfc-in_image_{}.png'.format(epoch))
    if loss.data[0] < 0.15: #arbitrary number because I saw that it works well enough
        break


model.save_model("2x2-2xfc-layer", "CAE")

## Preliminary Results 

Experiments with the following configurations:


## Pytorch understanding tests